<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>

# Climate Change Satellite Image Classification Competition Model Submission Guide - keras

---
**About the Original Data:**<br>
*Data and Description accessed from [Tensorflow](https://www.tensorflow.org/datasets/catalog/bigearthnet)* <br>
The BigEarthNet is a new large-scale Sentinel-2 benchmark archive, consisting of 590,326 Sentinel-2 image patches. The image patch size on the ground is 1.2 x 1.2 km with variable image size depending on the channel resolution. This is a multi-label dataset with 43 imbalanced labels, which has been simplified to single labels with 3 categories for the purposes of this competition.

To construct the BigEarthNet, 125 Sentinel-2 tiles acquired between June 2017 and May 2018 over the 10 countries (Austria, Belgium, Finland, Ireland, Kosovo, Lithuania, Luxembourg, Portugal, Serbia, Switzerland) of Europe were initially selected. All the tiles were atmospherically corrected by the Sentinel-2 Level 2A product generation and formatting tool (sen2cor). Then, they were divided into 590,326 non-overlapping image patches. Each image patch was annotated by the multiple land-cover classes (i.e., multi-labels) that were provided from the CORINE Land Cover database of the year 2018 (CLC 2018).

Bands and pixel resolution in meters:

    B01: Coastal aerosol; 60m
    B02: Blue; 10m
    B03: Green; 10m
    B04: Red; 10m
    B05: Vegetation red edge; 20m
    B06: Vegetation red edge; 20m
    B07: Vegetation red edge; 20m
    B08: NIR; 10m
    B09: Water vapor; 60m
    B11: SWIR; 20m
    B12: SWIR; 20m
    B8A: Narrow NIR; 20m

License: Community Data License Agreement - Permissive, Version 1.0."

**Competition Data Specifics:**<br>
For the purpose of this competition, the original BigEarthNet dataset has been simplified to 20,000 images (15,000 training images and 5,000 test images) with 3 categories: "forest", "nonforest", and "snow_shadow_cloud", which contains images of snow and clouds. <br>
Each "image" is a folder with 12 satellite image layers, each of which pics up on different features. The example preprocessor uses just three layers: B02, B03, and B04, which contain the standard RGB layers used in ML models. However, you are free to use any combination of the satellite image layers. 

**Data Source:**<br>
Sumbul, G, Charfuelan, M, Demir, B and Markl, V. (2019). BigEarthNet: A Large-Scale Benchmark Archive For Remote Sensing Image Understanding. *Computing Research Repository (CoRR), abs/1902.06148.* https://www.tensorflow.org/datasets/catalog/bigearthnet




# Overview
---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Load Data

In [ ]:
#install aimodelshare library
! pip install aimodelshare-nightly

In [2]:
# Get competition data - May take a couple minutes due to size of data set
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Unzip Data - May take a couple minutes due to size of data set
import zipfile
with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
    zip_ref.extractall('competition_data')

##2.   Preprocess data / Write and Save Preprocessor function


In [4]:
# Set up for data preprocessing
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(imageband_directory):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            imageband_directory
                path to folder with 13 satellite image bands
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import PIL
        import os
        import numpy as np
        import tensorflow_datasets as tfds

        def _load_tif(data):
            """Loads TIF file and returns as float32 numpy array."""
            img=tfds.core.lazy_imports.PIL_Image.open(data)
            img = np.array(img.getdata()).reshape(img.size).astype(np.float32)
            return img

        image_list = []
        filelist1=os.listdir(imageband_directory)
        for fpath in filelist1:
          fullpath=imageband_directory+"/"+fpath
          if fullpath.endswith(('B02.tif','B03.tif','B04.tif')):
              imgarray=_load_tif(imageband_directory+"/"+fpath)
              image_list.append(imgarray)

        X = np.stack(image_list,axis=2)   # to get (height,width,3)

        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        X=X/18581 # min max transform to max value
        return X

In [6]:
# Create complete list of file names
forestfilenames=["competition_data/trainingdata/forest/"+x for x in os.listdir("competition_data/trainingdata/forest")]
nonforestfilenames=["competition_data/trainingdata/nonforest/"+x for x in os.listdir("competition_data/trainingdata/nonforest")]
otherfilenames=["competition_data/trainingdata/other/"+x for x in os.listdir("competition_data/trainingdata/other")]

filenames=forestfilenames+nonforestfilenames+otherfilenames

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in filenames:
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass  
  

In [7]:
# Set up y data
from itertools import repeat
forest=repeat("forest",5000)
nonforest=repeat("nonforest",5000)
other=repeat("snow_shadow_cloud",5000)
ylist=list(forest)+list(nonforest)+list(other)

In [8]:
# Shuffle X and y data
from sklearn.utils import shuffle
X_train, y_train = shuffle(preprocessed_image_data, ylist, random_state=0)

In [9]:
X_train=np.vstack(X_train) # convert X from list to array

In [10]:
X_train.shape

(15000, 120, 120, 3)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

In [12]:
# Run using GPU
with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=[120, 120, 3]))
    model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(16, activation='relu')) # One fully-connected layer of 16 neurons.
    model.add(Dropout(0.2))

    model.add(Dense(3, activation='softmax')) # Last fully-connected layer of 3 outputs (3 categories).

    # Compile model...
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    import pandas
    model.fit(X_train, pandas.get_dummies(y_train),
                validation_split = .3, 
                epochs = 10)


Epoch 1/10
329/329 [==============================] - 243s 735ms/step - loss: 0.7915 - accuracy: 0.6098 - val_loss: 0.7060 - val_accuracy: 0.5849
Epoch 2/10
329/329 [==============================] - 242s 734ms/step - loss: 0.7134 - accuracy: 0.6282 - val_loss: 0.7069 - val_accuracy: 0.6160
Epoch 3/10
329/329 [==============================] - 250s 759ms/step - loss: 0.7021 - accuracy: 0.6316 - val_loss: 0.6983 - val_accuracy: 0.6351
Epoch 4/10
329/329 [==============================] - 243s 738ms/step - loss: 0.6931 - accuracy: 0.6290 - val_loss: 0.6765 - val_accuracy: 0.6122
Epoch 5/10
329/329 [==============================] - 241s 733ms/step - loss: 0.6782 - accuracy: 0.6350 - val_loss: 0.6696 - val_accuracy: 0.6222
Epoch 6/10
329/329 [==============================] - 243s 738ms/step - loss: 0.6836 - accuracy: 0.6309 - val_loss: 0.7558 - val_accuracy: 0.5422
Epoch 7/10
329/329 [==============================] - 242s 735ms/step - loss: 0.6693 - accuracy: 0.6414 - val_loss: 0.7283 -

#### Save preprocessor function to local "preprocessor.zip" file

In [13]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

can't pickle module objects
Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [14]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [15]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers=[str(x) for x in range(1, 5001)]
filenames=["competition_data/testdata/test/test"+x for x in filenumbers]

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in filenames:
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass  

In [16]:
X_test=np.vstack(preprocessed_image_data) 

In [17]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [18]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [19]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [pandas.get_dummies(y_train).columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [20]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,deep_learning,model_type,depth,num_params,dropout_layers,dense_layers,flatten_layers,conv2d_layers,maxpooling2d_layers,softmax_act,relu_act,loss,optimizer,model_config,memory_size,username,version
0,54.28%,46.12%,44.40%,48.11%,sklearn,nan,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,AIModelShare,2
1,56.88%,45.37%,42.31%,49.11%,sklearn,nan,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,AIModelShare,3
2,45.60%,43.39%,46.70%,40.53%,keras,True,Sequential,8.000000,1847811.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,3.000000,str,RMSprop,"{'name': 'sequential', 'layers...",2114288.000000,AIModelShare,5
3,40.08%,43.37%,44.80%,47.78%,keras,True,Sequential,8.000000,1847811.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,3.000000,str,RMSprop,"{'name': 'sequential', 'layers...",2233032.000000,AIModelShare,1
4,34.88%,32.55%,34.93%,35.20%,unknown,nan,unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None...,nan,AIModelShare,4


## 5. Repeat submission process to improve place on leaderboard

*Train and submit your own models using code modeled after what you see above.*

It may also be useful to examine the architeture of models that perform particuarly well/poorly, or to compare models you've created with similar models submitted by others. Use the compare_models function in combination with the leaderboard to learn more about models that been previously submitted and potentially make decisiona about what you should do next.

In [23]:
# Compare two or more models
data=mycompetition.compare_models([1, 5], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_5_Layer,Model_5_Shape,Model_5_Params
0,Conv2D,"[None, 120, 120, 32]",416,Conv2D,"[None, 120, 120, 32]",416
1,Conv2D,"[None, 120, 120, 32]",4128,Conv2D,"[None, 120, 120, 32]",4128
2,MaxPooling2D,"[None, 60, 60, 32]",0,MaxPooling2D,"[None, 60, 60, 32]",0
3,Dropout,"[None, 60, 60, 32]",0,Dropout,"[None, 60, 60, 32]",0
4,Flatten,"[None, 115200]",0,Flatten,"[None, 115200]",0
5,Dense,"[None, 16]",1843216,Dense,"[None, 16]",1843216
6,Dropout,"[None, 16]",0,Dropout,"[None, 16]",0
7,Dense,"[None, 3]",51,Dense,"[None, 3]",51
